# CityBikes

Send a request to CityBikes for the city of your choice. 

## City Bike Approach

### Sandbox and POC 
* GET the list of ALL providers using the general link
* In the Main list there are several providers for each city (Vancovuer x3). 
* There are not specific details in the main list. 
* GET requests are required for each provider in the target city. 
* Each **detailed** request will have to be consolidated into a DataFrame


## Step 1 - Obtain Master List of City Bike providers

* Library required:
    * requests - for API GET
    * json - to import the json response 
    * pandas to manipulate the dataframes

In [15]:
import requests
import json
import pandas as pd

In [50]:
## Create a funtion to pull City Bike info. 
'''City is the only parameter for the call. The default will be null'''
def city_bike_api_get(provider_id=""):
    url = f"http://api.citybik.es/v2/networks/{provider_id}"
    payload = {}
    headers = {}
    print(url)
    
    response = requests.request("GET", url, headers=headers, data=payload)

    print(response.text)

    if response.status_code == 200:
        # Construct the filename with the index
        filename = f'output_citybike_ALL.json'
        
        # Assuming you want to save the data as 'output....json'
        with open(filename, 'w') as json_file:
            json.dump(response.json(), json_file, indent=4)
            print('JSON data saved successfully.')
    else:
        print(f"Error: Received status code {response.status_code}")

city_bike_api_get('mobibikes')


http://api.citybik.es/v2/networks/mobibikes
{"network":{"company":["Vanncouver Bike Share Inc.","CycleHop LLC","City of Vancouver","Shaw Communications Inc.","Fifteen"],"gbfs_href":"https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json","href":"/v2/networks/mobibikes","id":"mobibikes","location":{"city":"Vancouver","country":"CA","latitude":49.2827,"longitude":-123.1207},"name":"Mobi","stations":[{"empty_slots":29,"extra":{"ebikes":0,"has_ebikes":true,"last_updated":1695259099,"normal_bikes":4,"renting":true,"returning":true,"slots":36,"uid":"0001"},"free_bikes":4,"id":"7a19c49f486d7c0c02b3685d7b240448","latitude":49.262487,"longitude":-123.114397,"name":"10th & Cambie","timestamp":"2023-09-21T01:26:33.203000Z"},{"empty_slots":2,"extra":{"ebikes":4,"has_ebikes":true,"last_updated":1695259497,"normal_bikes":10,"renting":true,"returning":true,"slots":16,"uid":"0004"},"free_bikes":14,"id":"32603a87cfca71d0f7dfa3513bad69d5","latitude":49.274566,"longitude":-123.121817,"name":"Yaletown-Roundho

## Step 2 - Parse list to identify provider ID for Target City

* create an object within Python using the json response.
* Dont forget to NORMALIZE (ie flatten it). 

Json structure: 
* networks
    * company
    * href
    * id
    * location:
        * city: **Target City**
    * name

In [28]:
city_bike_json= response.json()

# Use pd.json_normalize to flatten the "networks" JSON array
city_bike_df = pd.json_normalize(city_bike_json['networks'], errors='ignore')
city_bike_df

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN,NaN
2,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
4,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,Nafplio,GR,37.563940,22.809340,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,[Instituto Tecnológico de Castilla y León (ITCL)],/v2/networks/albabici,albabici,AlbaBici,Albacete,ES,38.994300,-1.860200,NaN,NaN,NaN,NaN,NaN
586,[Instituto Tecnológico de Castilla y León (ITCL)],/v2/networks/bicialcazar,bicialcazar,BiciAlcázar,Alcázar de San Juan,ES,39.390100,-3.210100,NaN,NaN,NaN,NaN,NaN
587,[Instituto Tecnológico de Castilla y León (ITCL)],/v2/networks/tallinja-bike,tallinja-bike,Tallinja Bike,Valletta - Malta - Gozo,MT,35.899200,14.514100,NaN,NaN,NaN,NaN,NaN
588,"[PBSC Urban Solutions, Shift Transit]",/v2/networks/tugo-bike-share,tugo-bike-share,Tugo Bike Share,"Tucson, AZ",US,32.222500,-110.974900,NaN,https://tucson.publicbikesystem.net/customer/g...,NaN,NaN,NaN


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

## Step 3: Parse Main Directory for Target City

* Filter the list of network providers for one that is in the target city. 
* declare a variable for 'provider id' based on filter result
* use this variable the argument into a second API call to get the specific details on the provider. (overrides the default "")
*  

In [43]:
cities = city_bike_df[city_bike_df['location.city']=='Vancouver']
city_ids = cities['id']
city_ids 
    

210    mobibikes
Name: id, dtype: object

# Step 4: Use Provider ID to return detailed information. 
* use existing API call from above, but amend with the [city_id] in the URL suffix.
* Since there is only 1 provider in the target city, run the function as a standalone. 
* If there were multiple results, would nest it into a FOR LOOP to generate multiple api calls. 

In [44]:
city_bike_api_get(city_ids)

{"networks":[{"company":["\u0417\u0410\u041e \u00ab\u0421\u0438\u0442\u0438\u0411\u0430\u0439\u043a\u00bb"],"href":"/v2/networks/velobike-moscow","id":"velobike-moscow","location":{"city":"Moscow","country":"RU","latitude":55.75,"longitude":37.616667},"name":"Velobike"},{"company":["Urban Infrastructure Partner"],"href":"/v2/networks/baerum-bysykkel","id":"baerum-bysykkel","location":{"city":"B\u00e6rum","country":"NO","latitude":59.89455,"longitude":10.546343},"name":"Bysykkel"},{"company":["Comunicare S.r.l."],"href":"/v2/networks/bicincitta-siena","id":"bicincitta-siena","location":{"city":"Siena","country":"IT","latitude":43.3186,"longitude":11.3306},"name":"Bicincitt\u00e0","source":"https://www.bicincitta.com/frmLeStazioni.aspx?ID=202"},{"company":["Cyclopolis Systems"],"href":"/v2/networks/cyclopolis-maroussi","id":"cyclopolis-maroussi","location":{"city":"Maroussi","country":"GR","latitude":38.0568722388,"longitude":23.8083299536},"name":"Cyclopolis"},{"company":["Cyclopolis Sy

Put your parsed results into a DataFrame.